In [1]:
%matplotlib qt

In [2]:
# MEG files
%ll camcan/BIDSsep/passive

total 0
drwxr-x---@ 4 antoinecollas  staff  128 Nov 10 15:33 sub-CC220352/
drwxr-x---@ 3 antoinecollas  staff   96 Nov 10 16:11 sub-CC610631/
drwxr-x---@ 3 antoinecollas  staff   96 Nov  4 17:36 sub-CC720986/
drwxr-x---@ 3 antoinecollas  staff   96 Nov  4 17:37 sub-CC721292/
drwxr-x---@ 3 antoinecollas  staff   96 Nov  4 17:38 sub-CC721519/
drwxr-x---@ 3 antoinecollas  staff   96 Nov  4 17:38 sub-CC721888/
drwxr-x---@ 3 antoinecollas  staff   96 Nov 10 16:06 sub-CC721894/
drwxr-x---@ 3 antoinecollas  staff   96 Nov 10 16:36 sub-CC723395/


In [3]:
# emptyroom files (no patient during the recording)
%ll camcan/emptyroom/

total 0
drwxr-x---@ 3 antoinecollas  staff  96 Nov 10 15:36 CC220352/
drwxr-x---@ 3 antoinecollas  staff  96 Nov 10 16:11 CC610631/
drwxr-x---@ 3 antoinecollas  staff  96 Nov  8 10:54 CC720986/
drwxr-x---@ 3 antoinecollas  staff  96 Nov  8 14:51 CC721519/
drwxr-x---@ 3 antoinecollas  staff  96 Nov 10 16:06 CC721894/
drwxr-x---@ 3 antoinecollas  staff  96 Nov 10 16:36 CC723395/


In [4]:
# transformation files (head-to-MRI)
%ll camcan/trans/

total 32
-rw-r--r--@ 1 antoinecollas  staff  212 Nov 10 15:31 sub-CC220352-trans.fif
-rw-r--r--@ 1 antoinecollas  staff  212 Nov 10 16:15 sub-CC610631-trans.fif
-rw-r--r--@ 1 antoinecollas  staff  212 Nov  8 14:48 sub-CC721519-trans.fif
-rw-r--r--@ 1 antoinecollas  staff  212 Nov 10 16:36 sub-CC723395-trans.fif


In [5]:
# MRI
%ll camcan/freesurfer/

total 0
drwxr-x---@ 9 antoinecollas  staff  288 Nov 10 15:39 CC220352/
drwxr-x---@ 9 antoinecollas  staff  288 Nov 10 16:12 CC610631/
drwxr-x---@ 9 antoinecollas  staff  288 Nov  8 15:12 CC721519/
drwxr-x---@ 9 antoinecollas  staff  288 Nov 10 16:00 CC721894/
drwxr-x---@ 9 antoinecollas  staff  288 Nov 10 16:36 CC723395/


In [6]:
# CALIBRATION file
%ll sss_cal.dat

-rw-r--r--@ 1 antoinecollas  staff  39114 Nov  4 17:31 sss_cal.dat


In [7]:
# CROSSTALK file
%ll ct_sparse.fif

-rw-r--r--@ 1 antoinecollas  staff  47399 Nov  4 17:31 ct_sparse.fif


# Load libraries and define paths

In [8]:
from autoreject import get_rejection_threshold
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import mne
from nilearn.plotting import plot_stat_map

PLOT = True
PLOT_3D = PLOT and False

# CC720986: good visual ERP tomo
SUBJECT = 'CC723395'  # CC220352, CC720986, CC721519
# DATA_PATH = '/storage/store/data/camcan/BIDSsep'
DATA_PATH = 'camcan/BIDSsep'
EMPTYROOM_PATH = 'camcan/emptyroom'
FREESURFER_PATH = 'camcan/freesurfer'
TRANS_PATH = 'camcan/trans'
CALIBRATION_PATH = 'sss_cal.dat'
CROSSTALK_PATH = 'ct_sparse.fif'

# Load raw data and empty room recordings

In [9]:
data_folder = os.path.join(DATA_PATH, 'passive', 'sub-' + SUBJECT, 'ses-passive')
filename = 'sub-' + SUBJECT + '_ses-passive_task-passive_meg.fif'
data_raw_file = os.path.join(data_folder, 'meg', filename)
raw = mne.io.read_raw_fif(data_raw_file, preload=True, verbose=False)

data_er_folder = os.path.join(EMPTYROOM_PATH, SUBJECT)
filename = 'emptyroom_' + SUBJECT + '.fif'
data_raw_er_file = os.path.join(data_er_folder, filename)
raw_er = mne.io.read_raw_fif(data_raw_er_file, preload=True, verbose=False)

/var/folders/gb/d2cp82c54_34yn2drlvfm9ww0000gn/T/ipykernel_43944/3845118813.py:9: RuntimeWarning: This filename (camcan/emptyroom/CC723395/emptyroom_CC723395.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_er = mne.io.read_raw_fif(data_raw_er_file, preload=True, verbose=False)


# Plot raw data and psd

In [10]:
if PLOT:
    raw.copy().pick(['meg']).plot(duration=1, start=40, scalings=2*1e-10, n_channels=5)
    plt.show()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


In [11]:
if PLOT:
    raw.compute_psd().plot()
    plt.show()

Effective window size : 0.256 (s)


# Find bad channels and maxfilter

In [12]:
raw_check = raw.copy()
auto_noisy_chs, auto_flat_chs, auto_scores = mne.preprocessing.find_bad_channels_maxwell(
    raw.copy(), cross_talk=CROSSTALK_PATH, calibration=CALIBRATION_PATH,
    return_scores=True, verbose=False)
raw.info['bads'] = auto_noisy_chs + auto_flat_chs
raw_er.info['bads'] = raw.info['bads']

In [13]:
raw.info['bads']

['MEG0612']

In [14]:
raw_sss = mne.preprocessing.maxwell_filter(
    raw.copy(), cross_talk=CROSSTALK_PATH, calibration=CALIBRATION_PATH, verbose=False)
raw_er_sss = mne.preprocessing.maxwell_filter(
    raw_er.copy(), cross_talk=CROSSTALK_PATH, calibration=CALIBRATION_PATH, coord_frame='meg', verbose=False)

In [15]:
# plt comparison
if PLOT:
    raw.copy().pick(['meg']).plot(duration=1.0, start=40.0, scalings=1e-9, n_channels=5)
    raw_sss.copy().pick(['meg']).plot(duration=1.0, start=40.0, scalings=1e-9, n_channels=5)
    raw.compute_psd().plot()
    raw_sss.compute_psd().plot()
    plt.show()

Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6
Effective window size : 0.256 (s)
Effective window size : 0.256 (s)


# Low pass filter

In [16]:
raw_sss.filter(l_freq=1, h_freq=30, verbose=False)
raw_er_sss.filter(l_freq=1, h_freq=30, verbose=False)
# raw_sss.notch_filter(np.arange(50, 201, 50))
raw_sss = raw_sss.crop(tmax=130)
if PLOT:
    raw_sss.copy().pick(['meg']).plot(duration=150.0, start=0.0, scalings=5*1e-11, n_channels=5)
    raw_sss.compute_psd().plot()
    plt.show()

Using pyopengl with version 3.1.6
Effective window size : 0.256 (s)


# Load events, create epochs and evoked

In [17]:
event_dict = {'auditory/300Hz': 6, 'auditory/600Hz': 7, 'auditory/1200Hz': 8, 'visual': 9}
events = mne.find_events(raw_sss, verbose=False)
if PLOT:
    fig = mne.viz.plot_events(
        events, event_id=event_dict, sfreq=raw_sss.info['sfreq'], first_samp=raw_sss.first_samp)
    plt.show()

In [18]:
# create epochs
TMIN, TMAX = -0.2, 0.5
epochs = mne.Epochs(
    raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), event_id=event_dict, preload=True, verbose=False)
epochs.get_data().shape

(102, 339, 701)

In [19]:
epochs.plot()

Using pyopengl with version 3.1.6


In [20]:
# reject some epochs
reject = get_rejection_threshold(epochs, verbose=False)
print(reject)
epochs = mne.Epochs(
    raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), reject=reject,
    event_id=event_dict, preload=True, verbose=False)
epochs.get_data().shape

{'mag': 2.0004186496515575e-12, 'grad': 7.130712618029352e-11, 'eog': 0.00047912599931976274}


(82, 339, 701)

In [21]:
evoked = dict()
for event in event_dict:
    evoked[event] = epochs[event].average()

In [22]:
EVENT = 'visual'
# EVENT = 'auditory'
if PLOT:
    print(EVENT)
    epochs[EVENT].average().plot()
    plt.show()

visual


In [23]:
all_times = np.linspace(0.1, 0.3, num=6)
if PLOT:
    evoked[EVENT].plot_topomap(all_times, ch_type='mag')
    plt.show()

# Compute noise covariance from baseline segments

In [24]:
noise_cov = mne.compute_covariance(epochs, tmax=0, method='auto', rank=None, verbose=False)
if PLOT:
    noise_cov.plot(raw_sss.info)
    plt.show()

Computing rank from covariance with rank=None
    Using tolerance 2.2e-14 (2.2e-16 eps * 102 dim * 0.95  max singular value)
    Estimated rank (mag): 73
    MAG: rank 73 computed from 102 data channels with 0 projectors
Computing rank from covariance with rank=None
    Using tolerance 2.2e-13 (2.2e-16 eps * 204 dim * 4.9  max singular value)
    Estimated rank (grad): 73
    GRAD: rank 73 computed from 204 data channels with 0 projectors


In [25]:
if PLOT:
    print(EVENT)
    evoked[EVENT].plot_white(noise_cov, time_unit='s', verbose=False)
    plt.show()

visual


# Compute noise covariance from empty room recordings

In [26]:
if PLOT:
    raw_er_sss.copy().pick(['meg']).plot(duration=150.0, start=0.0, scalings=5*1e-11, n_channels=5)
    plt.show()

Using pyopengl with version 3.1.6


In [27]:
noise_cov_er = mne.compute_raw_covariance(raw_er_sss, tmin=0, tmax=None, verbose=False)
if PLOT:
    print('Noise covariance matrix:')
    noise_cov.plot(raw_er.info, show_svd=False)
    print('Noise covariance matrix empty room:')
    noise_cov_er.plot(raw_er_sss.info, show_svd=False)
    plt.show()

Noise covariance matrix:
Noise covariance matrix empty room:


In [28]:
if PLOT:
    print(EVENT)
    evoked[EVENT].plot_white(noise_cov_er, time_unit='s', verbose=False)
    plt.show()

visual


/var/folders/gb/d2cp82c54_34yn2drlvfm9ww0000gn/T/ipykernel_43944/3374418727.py:3: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (74 > 73). Consider setting rank to "auto" or setting it explicitly as an integer.
  evoked[EVENT].plot_white(noise_cov_er, time_unit='s', verbose=False)
/var/folders/gb/d2cp82c54_34yn2drlvfm9ww0000gn/T/ipykernel_43944/3374418727.py:3: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (74 > 73). Consider setting rank to "auto" or setting it explicitly as an integer.
  evoked[EVENT].plot_white(noise_cov_er, time_unit='s', verbose=False)


# Compute the dSPM inverse solution on the cortical surface 

## Plot the coregistration

The coregistration is the operation that allows to position the head and the sensors in a common coordinate system.

In [29]:
trans_file = os.path.join(TRANS_PATH, 'sub-' + SUBJECT + '-trans.fif')
if PLOT_3D:
    mne.viz.plot_alignment(
        raw_sss.info, trans_file, subject=SUBJECT, dig=True,
        meg=['helmet', 'sensors'], subjects_dir=FREESURFER_PATH, surfaces='head-dense', verbose=False)

## Plot BEM and source space

bem = boundary element model

The BEM surfaces are the triangulations of the interfaces between different tissues needed for forward computation. These surfaces are for example the inner skull surface, the outer skull surface and the outer skin surface, a.k.a. scalp surface.

In [30]:
# src = mne.setup_source_space(SUBJECT, spacing='oct4', subjects_dir=FREESURFER_PATH, verbose=False)

SPHERE = np.array([0, -0.005, 0, 0.085])
surface_path = os.path.join(FREESURFER_PATH, SUBJECT, 'bem', 'inner_skull.surf')
src = mne.setup_volume_source_space(
    SUBJECT, surface=surface_path, subjects_dir=FREESURFER_PATH, sphere=SPHERE, verbose=False)

In [31]:
if PLOT:
    mne.viz.plot_bem(SUBJECT, subjects_dir=FREESURFER_PATH, src=src)
    plt.show()

Using surface: /Users/antoinecollas/Dropbox/postdoc/welcome_duty/camcan/freesurfer/CC723395/bem/inner_skull.surf
Using surface: /Users/antoinecollas/Dropbox/postdoc/welcome_duty/camcan/freesurfer/CC723395/bem/outer_skull.surf
Using surface: /Users/antoinecollas/Dropbox/postdoc/welcome_duty/camcan/freesurfer/CC723395/bem/outer_skin.surf


## Plot sources in 3D

In [32]:
PLOT_3D = True
if PLOT_3D:
    fig = mne.viz.plot_alignment(info=raw.info, trans=trans_file,
        subject=SUBJECT, subjects_dir=FREESURFER_PATH, surfaces='white', coord_frame='mri', src=src)
    mne.viz.set_3d_view(fig)

Using pyvistaqt 3d backend.

Getting helmet for system 306m
Channel types::	grad: 203, mag: 102


## Make bem model

In [33]:
# CONDUCTIVITY = (0.3, 0.006, 0.3)  # for three layers
CONDUCTIVITY = (0.3,)  # for single layer
model = mne.make_bem_model(SUBJECT, ico=4, conductivity=CONDUCTIVITY, subjects_dir=FREESURFER_PATH, verbose=False)
bem = mne.make_bem_solution(model, verbose=False)

In [34]:
fwd = mne.make_forward_solution(data_raw_file, trans=trans_file, src=src, bem=bem, verbose=False)
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 306 sensors x 32235 dipoles


In [35]:
inverse_operator = mne.minimum_norm.make_inverse_operator(raw_sss.info, fwd, noise_cov, verbose=False)

In [36]:
METHOD = 'dSPM'
print(EVENT)
stc = mne.minimum_norm.apply_inverse(evoked[EVENT], inverse_operator, method=METHOD, verbose=False)
if PLOT:
    stc.plot(src=src, subjects_dir=FREESURFER_PATH)

visual
Showing: t = 0.162 s, (6.0, -54.7, -26.7) mm, [18, 6, 15] vox, 18531 vertex
Using control points [ 5.40977085  6.7454156  16.26957795]


# Electroocoulogram

In [37]:
eog_evoked = mne.preprocessing.create_eog_epochs(raw_sss, verbose=False).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))  # subtract mean signal
if PLOT:
    eog_evoked.plot()
    plt.show()

Applying baseline correction (mode: mean)


# Electrocardiogram

In [38]:
ecg_evoked = mne.preprocessing.create_ecg_epochs(raw_sss, verbose=False).average()
ecg_evoked.apply_baseline(baseline=(None, -0.2))
ecg_evoked.plot()
plt.show()

Applying baseline correction (mode: mean)


# ICA

In [39]:
ica = mne.preprocessing.ICA(n_components=0.999, method='picard', max_iter='auto', random_state=123)
ica.fit(raw_sss)

Fitting ICA to data using 306 channels (please be patient, this may take a while)
Selecting by explained variance: 70 components
Fitting ICA took 59.2s.


Method,picard
Fit,430 iterations on raw data (130001 samples)
ICA components,70
Available PCA components,306
Channel types,"mag, grad"
ICA components marked for exclusion,—


In [40]:
if PLOT:
    ica.plot_components()
    plt.show()

/Users/antoinecollas/miniforge3/envs/welcome_duty/lib/python3.10/site-packages/mne/viz/_mpl_figure.py:2083: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(FigureClass=FigureClass, **kwargs)


In [41]:
BLINK_CHANNELS = [1]
HEARTBEAT_CHANNELS = [7, 14]
if PLOT:
    ica.plot_sources(raw_sss, picks=BLINK_CHANNELS+HEARTBEAT_CHANNELS)
    plt.show()

Creating RawArray with float64 data, n_channels=6, n_times=130001
    Range : 24000 ... 154000 =     24.000 ...   154.000 secs
Ready.
Using pyopengl with version 3.1.6


In [42]:
if PLOT:
    # blinks
    ica.plot_overlay(raw_sss, exclude=BLINK_CHANNELS, picks='mag')
    # heartbeats
    ica.plot_overlay(raw_sss, exclude=HEARTBEAT_CHANNELS, picks='mag')
    plt.show()

Applying ICA to Raw instance
    Transforming to ICA space (70 components)
    Zeroing out 1 ICA component
    Projecting back using 306 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (70 components)
    Zeroing out 2 ICA components
    Projecting back using 306 PCA components


In [43]:
ica.exclude = BLINK_CHANNELS+HEARTBEAT_CHANNELS
reconst_raw_sss = raw_sss.copy()
ica.apply(reconst_raw_sss)

Applying ICA to Raw instance
    Transforming to ICA space (70 components)
    Zeroing out 3 ICA components
    Projecting back using 306 PCA components


<Raw | sub-CC723395_ses-passive_task-passive_meg.fif, 339 x 130001 (130.0 s), ~342.6 MB, data loaded>

In [44]:
epochs = mne.Epochs(
    raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), reject=reject,
    event_id=event_dict, preload=True, verbose=False)
evoked = dict()
for event in event_dict:
    evoked[event] = epochs[event].average()
if PLOT:
    print(EVENT)
    evoked[EVENT].plot()
    plt.show()

epochs = mne.Epochs(
    reconst_raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), reject=reject,
    event_id=event_dict, preload=True, verbose=False)
evoked = dict()
for event in event_dict:
    evoked[event] = epochs[event].average()
if PLOT:
    print(EVENT)
    evoked[EVENT].plot()
    plt.show()
all_times = np.linspace(0.1, 0.3, num=6)
if PLOT:
    evoked[EVENT].plot_topomap(all_times, ch_type='mag')
    plt.show()

visual
visual
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
['MEG0612']
Channels marked as bad:
none
